<a href="https://colab.research.google.com/github/venugopals002/data-science-portfolio/blob/master/Pacakging_material_consumption_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [80]:
df = pd.read_excel('/content/import file.xlsx')

In [ ]:
df.describe()

In [82]:
df['Coil_OD'] = (np.sqrt(((df['BATCH WT']*1000)/(0.00000785*math.pi*df['FG WIDTH'])*4)+(df['COIL INNER DIAMETER'])**2))*1.0052

In [ ]:
df.describe()

In [ ]:
df.head()


In [85]:
df['VCI_theory_Width'] = df['FG WIDTH']+(df['Coil_OD']-df['COIL INNER DIAMETER'])+200

In [ ]:
df.round().head()

In [87]:
# df['VCI_actual_width'] = 

In [88]:
# function for caculating VCI actual Width
def VCI_actual_width(df):
  if df['VCI_theory_Width'] >= 0 and df['VCI_theory_Width'] <=1830:
    return 1830
  elif df['VCI_theory_Width'] > 1830 and df['VCI_theory_Width'] <=2130:
    return 2130
  elif df['VCI_theory_Width'] > 2130 and df['VCI_theory_Width'] <=2740:
    return 2740
  else:
    return 2741
df['VCI_actual_width'] = df.apply(VCI_actual_width, axis=1)

In [ ]:
df.describe()

In [90]:
df['VCI_top_cover'] = (df['VCI_actual_width']*(df['Coil_OD']*math.pi+150))/1000000

In [ ]:
df.head()

In [92]:
df['LDPE_top_cover'] = (df['VCI_actual_width']*(df['Coil_OD']*math.pi+150))/1000000

In [ ]:
df.round().describe()

In [94]:
df['Angle_Board_OD'] = ((df['Coil_OD']*math.pi+150)*2)/1000

In [95]:
# function for calculating angle board ID consumption
def Angle_Board_ID(df):
  if df['COIL INNER DIAMETER'] == 508:
    return (1550 * 2)/1000
  elif df['COIL INNER DIAMETER'] == 610:
    return (1850*2)/1000

df['Angle_Board_ID'] = df.apply(Angle_Board_ID, axis=1)

In [ ]:
df.head()

In [97]:
df['BOPP_tape'] = (((1+1+6+32)*200)/1000)/65

In [98]:
df['5"_HDPE_tape'] = ((df['VCI_actual_width']+((df['COIL INNER DIAMETER']*3.14+150)*2)+df['FG WIDTH']+((df['Coil_OD']*3.14+150)*2)+((df['COIL INNER DIAMETER']*3.14+150)*2))/1000)/50
# F2 = ID, G2=OD, i2 = VCI actual Width, C2 = coil width 

In [ ]:
df.head()

In [100]:
# Function for Calculating VCI Inner Sleeve
def VCI_Inner_Cons(df):
  if df['COIL INNER DIAMETER'] == 508:
    return 2.08
  else:
    return 2.31

df['VCI_Inner'] = df.apply(VCI_Inner_Cons, axis=1)

In [101]:
df['Filament_tape'] = (((6*200)+(df['FG WIDTH']*2)+(6*200)+df['FG WIDTH'])/1000)/50
# C2 = Width

In [102]:
# Function for calculating Metwrapp side Disk Size
# ((IF(AND(Coil_OD <1250),"1250",IF(AND(Coil_OD>1250,G2<1550),"1550",IF(AND(Coil_OD>1550),"1850")))))
def Metwrapp_SD_Size(df):
  if df['Coil_OD'] <= 1250:
    return 1250
  elif df['Coil_OD'] > 1250 and df['Coil_OD'] < 1550:
    return 1550
  else:
    return 1850

df['Metwrapp_Side_Disk_size'] = df.apply(Metwrapp_SD_Size, axis=1)

In [ ]:
df.head()

In [104]:
# (((IF(AND(G2<1250),"1250",IF(AND(G2>1250,G2<1550),"1550",IF(AND(G2>1550),"1850")))))*2)/1000
# G2 = Coil_OD

# Function for calculating Metwrapp side Disk Consumption
def Metwrapp_SD_Consumption(df):
  if df['Coil_OD'] <= 1250:
    return (1250*2)/1000
  elif df['Coil_OD'] > 1250 and df['Coil_OD'] < 1550:
    return (1550*2)/1000
  else:
    return (1850*2)/1000

df['Metwrapp_side_Disk_Consumption'] = df.apply(Metwrapp_SD_Consumption, axis=1)

In [105]:
# =(1220*(F2*3.14+150))/1000000
# F2 = Coil ID

df['Fluteboard_Consumption'] = (1220*(df['COIL INNER DIAMETER']*3.14+150))/1000000

In [106]:
# =3*((G2*3.14+150)*0.64*32*0.00000785)
# G2 = Coil OD

df['Cir_Steel_Strap_Consumption'] = 3*((df['Coil_OD']*3.14+150)*0.64*32*0.00000785)

In [107]:
# Function for Calculating Eye Strap Consumption
# =(IF(AND(E2="Export Economical"),"6","4"))*((C2*2+(G2-F2)+150)*0.64*32*0.00000785)
# E2 = Pack Type, C2 = Coil Width, G2 = Coil OD, F2 = Coil ID
def Eye_Strap_Consumption(df):
  if df['NEW PACK CONFIGURATION'] == 'Export Economical':
    return (6*((df['FG WIDTH']*2+(df['Coil_OD']-df['COIL INNER DIAMETER'])+150)*0.64*32*0.00000785))
  else:
    return (4*((df['FG WIDTH']*2+(df['Coil_OD']-df['COIL INNER DIAMETER'])+150)*0.64*32*0.00000785))

df['Eye_Strap_Consumption'] = df.apply(Eye_Strap_Consumption, axis=1)

In [108]:
df['Total_Strap_Consumption'] = df['Cir_Steel_Strap_Consumption']+df['Eye_Strap_Consumption']

In [109]:
# Function to calculate Seals consumption
# =(IF(AND(E2="Export Economical"),"6","4"))+3

def Seals_Consumption(df):
  if df['NEW PACK CONFIGURATION'] == 'Export Economical':
    return 6+3
  else:
    return 4+3

df['Seals_Consumption'] = df.apply(Seals_Consumption, axis=1)

In [111]:
import time

TodaysDate = time.strftime("%d-%m-%Y")
Packaging_Material_Consumption = TodaysDate+".xlsx"

df.to_excel(Packaging_Material_Consumption, sheet_name='sheet1', index=False)

In [113]:
VCI_Consumption = df.groupby('VCI_actual_width')['VCI_top_cover'].sum()

In [ ]:
VCI_Consumption